In [ ]:
### This repo contain feature selection of all the Modalities, later on we have selected fwe modalties.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
# ^^^ pyforest auto-imports - don't write above this line
import pyforest
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pandas as pd

## Importing data

In [6]:
# df00=pd.read_csv("DS3-0.csv")
# df1=pd.read_csv("DS3-1.csv")
# df6=pd.read_csv("DS3-6.csv")
# df7=pd.read_csv("DS3-7.csv")
# df8=pd.read_csv("DS3-8.csv")
# df9=pd.read_csv("BookNHY.csv")
df00=pd.read_csv("Data_Resample.csv")

In [7]:
df00

,SrcBytes,DstBytes,SrcLoad,DstLoad,SrcGap,DstGap,SIntPkt,DIntPkt,SIntPktAct,DIntPktAct,...,Packet_num,Temp,SpO2,Pulse_Rate,SYS,DIA,Heart_rate,Resp_Rate,ST,Label
0,496,186,276914.00000,92305.000000,0,0,3.582333,1.901500,0.0,0,...,1,28.900000,0,0,0,0,0,0,0.000000,0
1,496,186,230984.00000,76995.000000,0,0,4.294667,2.901500,0.0,0,...,2,28.900000,0,0,0,0,78,17,0.400000,0
2,496,186,235723.00000,78574.000000,0,0,4.208333,2.863500,0.0,0,...,5,28.900000,89,101,0,0,79,17,0.400000,0
3,496,186,191284.00000,63761.000000,0,0,5.186000,3.803000,0.0,0,...,6,28.900000,91,103,0,0,79,17,0.400000,0
4,496,186,217958.00000,72653.000000,0,0,4.551333,2.841000,0.0,0,...,7,28.900000,91,103,0,0,82,17,0.400000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18934,496,186,224604.91090,74867.970300,0,0,4.416655,3.027166,0.0,0,...,1224,27.000000,98,79,129,76,81,33,0.200000,1
18935,496,186,249793.81530,83264.298220,0,0,3.971276,2.483794,0.0,0,...,13520,26.684123,98,73,148,84,73,19,0.300000,1
18936,496,186,16558.37307,5519.734424,0,0,59.919683,66.000302,0.0,0,...,15634,27.100000,96,73,148,84,73,19,0.300000,1
18937,496,186,194779.02260,64926.674220,0,0,5.092947,2.895792,0.0,0,...,11789,27.860793,98,73,148,84,73,19,0.300000,1


In [12]:
df11=pd.merge(df1,df9, how="inner" ,on = 'PATNO')
df11

,PATNO,BLDLAB_max,BLDDNA_max,PLASBFCT_max,PDMEDYN_max,UASPEC_max,UT1SPNRT_max,UT1SPNDR_max,UT1CFRG_max,BLDRNA_max,...,CSFCFRG_var,SMPLOCAL_var,WBCUNITB_var,TGLCRSLT_var,FLUORO_var,SPFI_var,BLDDNA_std,DurUT1SPNTM_std,DurSERSPNTM_std,NHY
0,3001.0,1.0,1.0,1.0,1.0,1.0,2500.0,15.0,6.0,1.0,...,41.200000,0.000000,0.0,15.466667,0.000000,0.000000,0.0,8.712061,9.968283,2
1,3002.0,1.0,1.0,1.0,1.0,1.0,2500.0,15.0,5.0,1.0,...,5.866667,0.000000,0.0,8.400000,0.000000,0.000000,0.0,3.507136,4.262237,2
2,3003.0,1.0,1.0,1.0,1.0,1.0,2500.0,15.0,5.0,1.0,...,6.000000,0.166667,0.0,4.566667,0.000000,0.000000,0.0,14.487926,8.221922,3
3,3010.0,1.0,1.0,1.0,1.0,1.0,2500.0,15.0,6.0,1.0,...,6.000000,0.000000,0.0,203.600000,0.000000,0.000000,0.0,31.780497,4.501851,2
4,3012.0,1.0,1.0,1.0,1.0,1.0,2500.0,15.0,5.0,1.0,...,0.300000,0.000000,0.0,5.466667,0.000000,0.000000,0.0,11.135529,5.278889,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236.0,1.0,1.0,1.0,0.0,1.0,2500.0,15.0,22.0,1.0,...,1.200000,0.000000,0.0,7.500000,0.000000,0.000000,0.0,28.533606,44.205957,0
1055,90456.0,1.0,1.0,1.0,0.0,1.0,2500.0,15.0,4.0,1.0,...,0.000000,0.166667,0.0,0.000000,0.166667,0.166667,0.0,8.518607,3.544949,2
1056,91097.0,1.0,1.0,1.0,0.0,1.0,2500.0,15.0,4.0,1.0,...,5.500000,0.166667,0.0,46.966667,0.000000,0.000000,0.0,8.931219,2.041241,0
1057,92490.0,1.0,1.0,1.0,0.0,1.0,2500.0,15.0,4.0,1.0,...,13.066667,0.000000,0.0,0.666667,0.166667,0.000000,0.0,26.445542,21.753161,2


In [13]:
df66=pd.merge(df6,df9, how="inner" ,on = 'PATNO')
df66

,PATNO,DFSTROKE_max,DFRSKFCT_max,DFPRESNT_max,DFRPROG_max,DFSTATIC_max,DFHEMPRK_max,DFAGESX_max,DFOTHCRS_max,DFRTREMP_max,...,DIASUP_var,HRSUP_var,SYSSTND_var,DIASTND_var,HRSTND_var,WGTKG_var,HTCM_var,TEMPC_var,DFRTREMA_std,NHY
0,3001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,62.700000,36.566667,148.266667,31.866667,51.366667,3.537667,1.866667,0.320000,0.000000,2
1,3002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,61.466667,32.800000,109.366667,66.400000,4.266667,4.214667,1.500000,0.246667,0.408248,2
2,3003.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,43.600000,53.100000,271.866667,56.000000,203.100000,48.014667,0.300000,0.022667,0.000000,3
3,3010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,5.066667,29.366667,189.366667,16.566667,133.866667,29.542667,0.266667,0.060000,0.000000,2
4,3012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,14.666667,19.866667,89.366667,11.500000,8.966667,4.086667,1.066667,0.057667,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.666667,45.366667,150.966667,7.866667,68.300000,3.600000,0.266667,0.486667,0.516398,0
1055,90456.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,17.866667,14.000000,105.366667,75.466667,57.200000,0.124000,4.166667,0.062667,0.000000,2
1056,91097.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,71.600000,75.866667,23.066667,66.666667,35.866667,2.911000,2.566667,0.058667,0.408248,0
1057,92490.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,32.700000,5.766667,1.866667,24.400000,44.966667,2.577667,0.000000,0.121667,0.408248,2


In [14]:
df77=pd.merge(df7,df9, how="inner" ,on = 'PATNO')
df77

,PATNO,NUPSOURC1a_max,NP1COG_max,NP1HALL_max,NP1DPRS_max,NP1ANXS_max,NP1APAT_max,NP1DDS_max,NP1SLPN_max,NP1SLPD_max,...,NP3HMOVL_var,NP3PRSPR_var,NP3PRSPL_var,NP3TTAPR_var,NP3LGAGL_var,NP3POSTR_var,NP3BRADY_var,NP3RTCON_var,MSEADLG_var,NHY
0,3001,1,1,0,1,1,1,0,1,2,...,0.166667,0.166667,0.566667,0.166667,0.666667,0.300000,0.300000,1.466667,6.666667,2
1,3002,1,2,1,3,2,2,1,3,2,...,0.000000,0.300000,0.166667,0.266667,0.266667,0.166667,0.166667,1.600000,17.500000,2
2,3003,1,1,0,0,2,0,0,3,2,...,0.266667,0.166667,0.300000,0.300000,0.166667,0.266667,0.000000,1.200000,7.500000,3
3,3010,1,1,1,3,1,2,1,4,3,...,0.400000,0.400000,0.300000,0.800000,0.566667,0.166667,0.166667,2.266667,44.166667,2
4,3012,1,2,1,1,1,3,0,1,2,...,0.266667,0.166667,0.266667,0.166667,0.400000,0.566667,0.166667,0.266667,14.166667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236,1,0,0,0,0,0,0,2,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0
1055,90456,1,2,0,1,1,1,0,1,2,...,0.300000,0.266667,0.166667,0.300000,0.166667,0.300000,0.300000,0.166667,24.166667,2
1056,91097,1,0,0,0,0,0,0,1,1,...,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0
1057,92490,1,1,0,0,0,2,0,2,2,...,0.166667,0.266667,0.166667,0.800000,0.266667,0.166667,0.666667,0.400000,26.666667,2


In [15]:
df88=pd.merge(df8,df9, how="inner" ,on = 'PATNO')
df88

,PATNO,GDSSATIS_max,GDSDROPD_max,GDSEMPTY_max,GDSBORED_max,GDSGSPIR_max,GDSAFRAD_max,GDSHAPPY_max,GDSHLPLS_max,GDSHOME_max,...,MCATOT_var,VLTANIM_var,VLTVEG_var,VLTFRUIT_var,AGE_ASSESS_SFTANIM_var,ESS1_var,ESS2_var,ESS5_var,UPSITBK4_std,NHY
0,3001.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.266667,16.666667,3.766667,3.766667,3.500000,0.166667,0.400000,0.166667,0.0,2
1,3002.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.900000,5.900000,10.966667,4.000000,3.500000,0.166667,0.666667,0.700000,0.0,2
2,3003.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.200000,1.600000,2.266667,3.866667,3.500000,0.166667,0.166667,0.266667,0.0,3
3,3010.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.500000,10.166667,3.066667,8.800000,4.800000,0.566667,0.666667,0.566667,0.0,2
4,3012.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,6.700000,3.200000,3.766667,2.800000,2.666667,0.166667,0.266667,0.666667,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,3.500000,9.600000,1.366667,1.600000,3.500000,0.300000,0.166667,0.166667,0.0,0
1055,90456.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,10.966667,0.566667,4.000000,7.466667,3.500000,0.400000,0.166667,0.266667,0.0,2
1056,91097.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,3.866667,3.900000,3.200000,3.466667,3.500000,0.266667,0.166667,0.166667,1.0,0
1057,92490.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,2.166667,2.400000,5.766667,0.400000,3.466667,0.166667,0.266667,0.966667,0.0,2


# FEATURES SELECTION

In [ ]:
from featurewiz import featurewiz

# DF0

In [17]:
features0 = featurewiz(df00, target='NHY', corr_limit=0.70, verbose=2)
features0

Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
Shape of your Data Set loaded: (1059, 28)
File not able to be loaded
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    27 Predictors classified...
        7 variable(s) removed since they were ID or low-information variables
Shape of your Data Set loaded: (1059, 28)
Number of processors on machine = 1
    No GPU active on this device
    Running XGBoost using CPU parameters
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    27 Predictors classified...
        7 variable(s) removed since they were ID or low-information variables
#### Single_Label Multi_Classification Feature Selection Started ####
Searching for highly correlated variables from 20 variables using SULOV method
#####  SULOV : Searching for Uncorrelated Li

(['REFERRAL',
  'HISPLAT',
  'PATAU',
  'MATAU',
  'RAWHITE',
  'Age',
  'PAGPARPD',
  'EDUCYRS',
  'FULSIB',
  'PATAUPD',
  'RANOS',
  'RAASIAN',
  'RAINDALS',
  'RABLACK',
  'GENDER',
  'HANDED'],
       REFERRAL  HISPLAT  PATAU  MATAU  RAWHITE  Age  PAGPARPD  EDUCYRS  \
 0           60        0      7      5        1   69         0       16   
 1            1        0     11      2        1   72         0       16   
 2            1        0      1      1        1   61         0       16   
 3           16        0      4      4        1   51         0       16   
 4            1        0      0      1        1   62         0       16   
 ...        ...      ...    ...    ...      ...  ...       ...      ...   
 1054         1        0      2      2        0   67         0       14   
 1055        99        0      0      3        1   84         0       18   
 1056        71        1      1      5        0   68         0       16   
 1057         1        0      4      2        1   7

In [10]:
# Making DAtaframe of Selected columns
columns =['PATNO','REFERRAL',
  'HISPLAT',
  'PATAU',
  'MATAU',
  'RAWHITE',
  'Age',
  'PAGPARPD',
  'EDUCYRS',
  'FULSIB',
  'PATAUPD',
  'RANOS',
  'RAASIAN',
  'RAINDALS',
  'RABLACK',
  'GENDER',
  'HANDED','NHY']
df0= pd.DataFrame(df00, columns=columns)
df0

<IPython.core.display.Javascript object>

,PATNO,REFERRAL,HISPLAT,PATAU,MATAU,RAWHITE,Age,PAGPARPD,EDUCYRS,FULSIB,PATAUPD,RANOS,RAASIAN,RAINDALS,RABLACK,GENDER,HANDED,NHY
0,3001,60,0,7,5,1,69,0,16,1,0,0,0,0,0,2,2,1
1,3002,1,0,11,2,1,72,0,16,2,0,0,0,0,0,1,1,2
2,3003,1,0,1,1,1,61,0,16,2,0,0,0,0,0,1,1,2
3,3010,16,0,4,4,1,51,0,16,2,0,0,0,0,0,2,1,2
4,3012,1,0,0,1,1,62,0,16,2,0,0,0,0,0,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236,1,0,2,2,0,67,0,14,5,0,0,0,0,0,2,1,0
1055,90456,99,0,0,3,1,84,0,18,3,0,0,0,0,0,2,3,0
1056,91097,71,1,1,5,0,68,0,16,1,0,0,0,0,0,2,1,0
1057,92490,1,0,4,2,1,73,0,12,2,0,0,0,0,0,2,1,0


In [20]:
df0.to_csv('df0_features.csv', index = False)

# DF1

In [21]:
features1= featurewiz(df11, target='NHY', corr_limit=0.70, verbose=2)
features1

Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
Shape of your Data Set loaded: (1059, 106)
File not able to be loaded
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    105 Predictors classified...
        2 variable(s) removed since they were ID or low-information variables
Shape of your Data Set loaded: (1059, 106)
Number of processors on machine = 1
    No GPU active on this device
    Running XGBoost using CPU parameters
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    105 Predictors classified...
        2 variable(s) removed since they were ID or low-information variables
#### Single_Label Multi_Classification Feature Selection Started ####
Searching for highly correlated variables from 103 variables using SULOV method
#####  SULOV : Searching for Uncorrelat

(['PDMEDYN_max',
  'PDMEDYN_min',
  'PLASBFCT_max',
  'UT1SPNRT_max',
  'UASPEC_min',
  'PATNO',
  'WBCRSLT_min',
  'DurSERSPNTM_std',
  'DurPLASSPNTM_min',
  'CSFSPNDR_var',
  'BSSPNRT_mean',
  'SMPLOCAL_max',
  'CSFCFRG_max',
  'RNASTTMP_var',
  'WBCRSLT_max',
  'BLDRNA_min',
  'WBCUNITB_min',
  'UT1SPNRT_min',
  'TGLCRSLT_var',
  'CSFCFRG_min',
  'BSVAFT_var',
  'DurPLASFFTM_mean',
  'BLDSER_mean',
  'CSFMETHD_min',
  'SMPLOCAL_var',
  'CSFCFRG_var',
  'DurUT1SPNTM_std',
  'DurPLASSPNTM_var',
  'LPSITE_var',
  'CSFNEEDL_var'],
       PDMEDYN_max  PDMEDYN_min  PLASBFCT_max  UT1SPNRT_max  UASPEC_min  \
 0             1.0          0.0           1.0        2500.0         0.0   
 1             1.0          0.0           1.0        2500.0         1.0   
 2             1.0          0.0           1.0        2500.0         1.0   
 3             1.0          0.0           1.0        2500.0         1.0   
 4             1.0          0.0           1.0        2500.0         1.0   
 ...          

In [22]:
# Making DAtaframe of Selected columns
columns =['PATNO','PDMEDYN_max',
  'PDMEDYN_min',
  'PLASBFCT_max',
  'UT1SPNRT_max',
  'UASPEC_min',
  'PATNO',
  'WBCRSLT_min',
  'DurSERSPNTM_std',
  'DurPLASSPNTM_min',
  'CSFSPNDR_var',
  'BSSPNRT_mean',
  'SMPLOCAL_max',
  'CSFCFRG_max',
  'RNASTTMP_var',
  'WBCRSLT_max',
  'BLDRNA_min',
  'WBCUNITB_min',
  'UT1SPNRT_min',
  'TGLCRSLT_var',
  'CSFCFRG_min',
  'BSVAFT_var',
  'DurPLASFFTM_mean',
  'BLDSER_mean',
  'CSFMETHD_min',
  'SMPLOCAL_var',
  'CSFCFRG_var',
  'DurUT1SPNTM_std',
  'DurPLASSPNTM_var',
  'LPSITE_var',
  'CSFNEEDL_var','NHY']
df1= pd.DataFrame(df11, columns=columns)
df1

<IPython.core.display.Javascript object>

,PATNO,PDMEDYN_max,PDMEDYN_min,PLASBFCT_max,UT1SPNRT_max,UASPEC_min,PATNO,WBCRSLT_min,DurSERSPNTM_std,DurPLASSPNTM_min,...,DurPLASFFTM_mean,BLDSER_mean,CSFMETHD_min,SMPLOCAL_var,CSFCFRG_var,DurUT1SPNTM_std,DurPLASSPNTM_var,LPSITE_var,CSFNEEDL_var,NHY
0,3001.0,1.0,0.0,1.0,2500.0,0.0,3001.0,0.0,9.968283,5.0,...,46.500000,0.833333,2.0,0.000000,41.200000,8.712061,99.366667,0.300000,0.000000,2
1,3002.0,1.0,0.0,1.0,2500.0,1.0,3002.0,0.0,4.262237,5.0,...,45.333333,0.833333,2.0,0.000000,5.866667,3.507136,83.500000,0.400000,1.500000,2
2,3003.0,1.0,0.0,1.0,2500.0,1.0,3003.0,0.0,8.221922,5.0,...,48.166667,1.000000,2.0,0.166667,6.000000,14.487926,141.766667,0.300000,0.000000,3
3,3010.0,1.0,0.0,1.0,2500.0,1.0,3010.0,0.0,4.501851,2.0,...,47.333333,1.000000,2.0,0.000000,6.000000,31.780497,93.100000,0.700000,0.000000,2
4,3012.0,1.0,0.0,1.0,2500.0,1.0,3012.0,0.0,5.278889,2.0,...,44.833333,1.000000,2.0,0.000000,0.300000,11.135529,114.400000,0.166667,0.166667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236.0,0.0,0.0,1.0,2500.0,1.0,85236.0,1.0,44.205957,20.0,...,98.333333,1.000000,1.0,0.000000,1.200000,28.533606,2167.500000,0.566667,0.166667,0
1055,90456.0,0.0,0.0,1.0,2500.0,1.0,90456.0,0.0,3.544949,21.0,...,48.833333,1.000000,1.0,0.166667,0.000000,8.518607,12.566667,0.000000,1.500000,2
1056,91097.0,0.0,0.0,1.0,2500.0,1.0,91097.0,1.0,2.041241,15.0,...,44.666667,1.000000,2.0,0.166667,5.500000,8.931219,4.166667,0.166667,0.000000,0
1057,92490.0,0.0,0.0,1.0,2500.0,1.0,92490.0,1.0,21.753161,17.0,...,55.000000,1.000000,1.0,0.000000,13.066667,26.445542,473.200000,0.666667,2.400000,2


In [30]:
df1.to_csv('df1_features.csv', index = False)

# DF6

In [23]:
features6= featurewiz(df66, target='NHY', corr_limit=0.70, verbose=2)
features6

Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
Shape of your Data Set loaded: (1059, 153)
File not able to be loaded
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    152 Predictors classified...
        8 variable(s) removed since they were ID or low-information variables
Shape of your Data Set loaded: (1059, 153)
Number of processors on machine = 1
    No GPU active on this device
    Running XGBoost using CPU parameters
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    152 Predictors classified...
        8 variable(s) removed since they were ID or low-information variables
#### Single_Label Multi_Classification Feature Selection Started ####
Searching for highly correlated variables from 144 variables using SULOV method
#####  SULOV : Searching for Uncorrelat

(['DFRTREMP_max',
  'DFRIGIDA_max',
  'DFBRADYA_min',
  'DFPGDIST_min',
  'DFFALLS_max',
  'DFPGDIST_var',
  'DFOTHCRS_max',
  'DFBRADYA_var',
  'DFRPROG_max',
  'DFBRADYP_var',
  'SENRLRSP_min',
  'DFPRESNT_max',
  'DFURDYS_max',
  'DFRIGIDP_var',
  'DFSTROKE_min',
  'DFRTREMP_var',
  'MSRLRSP_max',
  'DFRAPSPE_max',
  'DFURDYS_min',
  'CN8RSP_min',
  'DFOCULO_max',
  'DFBWLDYS_max',
  'DFFALLS_min',
  'DFATYP_min',
  'DFPATREM_min',
  'DFOTHPG_min',
  'DFCOGNIT_min',
  'DFDYSTON_min',
  'DFRTREMA_std',
  'BPARM_min',
  'CN1RSP_var',
  'SENLARSP_mean',
  'SYSSUP_var',
  'SYSSTND_var',
  'DIASTND_var',
  'COHSLRSP_mean',
  'WGTKG_var',
  'DFRIGIDA_var'],
       DFRTREMP_max  DFRIGIDA_max  DFBRADYA_min  DFPGDIST_min  DFFALLS_max  \
 0              1.0           0.0           0.0           0.0          0.0   
 1              1.0           0.0           0.0           0.0          1.0   
 2              0.0           0.0           0.0           0.0          0.0   
 3              1.0      

In [26]:
# Making DAtaframe of Selected columns
columns =['PATNO','DFRTREMP_max',
  'DFRIGIDA_max',
  'DFBRADYA_min',
  'DFPGDIST_min',
  'DFFALLS_max',
  'DFPGDIST_var',
  'DFOTHCRS_max',
  'DFBRADYA_var',
  'DFRPROG_max',
  'DFBRADYP_var',
  'SENRLRSP_min',
  'DFPRESNT_max',
  'DFURDYS_max',
  'DFRIGIDP_var',
  'DFSTROKE_min',
  'DFRTREMP_var',
  'MSRLRSP_max',
  'DFRAPSPE_max',
  'DFURDYS_min',
  'CN8RSP_min',
  'DFOCULO_max',
  'DFBWLDYS_max',
  'DFFALLS_min',
  'DFATYP_min',
  'DFPATREM_min',
  'DFOTHPG_min',
  'DFCOGNIT_min',
  'DFDYSTON_min',
  'DFRTREMA_std',
  'BPARM_min',
  'CN1RSP_var',
  'SENLARSP_mean',
  'SYSSUP_var',
  'SYSSTND_var',
  'DIASTND_var',
  'COHSLRSP_mean',
  'WGTKG_var',
  'DFRIGIDA_var','NHY']
df6= pd.DataFrame(df66, columns=columns)
df6

<IPython.core.display.Javascript object>

,PATNO,DFRTREMP_max,DFRIGIDA_max,DFBRADYA_min,DFPGDIST_min,DFFALLS_max,DFPGDIST_var,DFOTHCRS_max,DFBRADYA_var,DFRPROG_max,...,BPARM_min,CN1RSP_var,SENLARSP_mean,SYSSUP_var,SYSSTND_var,DIASTND_var,COHSLRSP_mean,WGTKG_var,DFRIGIDA_var,NHY
0,3001.0,1.0,0.0,0.0,0.0,0.0,0.266667,0.0,0.000000,0.0,...,1.0,0.000000,0.0,211.466667,148.266667,31.866667,0.000000,3.537667,0.000000,2
1,3002.0,1.0,0.0,0.0,0.0,1.0,0.166667,0.0,0.000000,0.0,...,1.0,0.000000,0.0,128.166667,109.366667,66.400000,0.000000,4.214667,0.000000,2
2,3003.0,0.0,0.0,0.0,0.0,0.0,0.166667,1.0,0.000000,0.0,...,1.0,0.000000,0.0,296.300000,271.866667,56.000000,0.000000,48.014667,0.000000,3
3,3010.0,1.0,0.0,0.0,0.0,0.0,0.300000,0.0,0.000000,0.0,...,1.0,0.000000,0.0,122.266667,189.366667,16.566667,0.000000,29.542667,0.000000,2
4,3012.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,...,1.0,0.000000,0.0,207.900000,89.366667,11.500000,0.333333,4.086667,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.266667,0.0,...,1.0,0.666667,0.0,80.666667,150.966667,7.866667,0.000000,3.600000,0.266667,0
1055,90456.0,1.0,1.0,0.0,0.0,0.0,0.266667,0.0,0.166667,0.0,...,1.0,0.266667,0.0,65.866667,105.366667,75.466667,0.166667,0.124000,0.166667,2
1056,91097.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.0,...,1.0,0.000000,0.0,82.266667,23.066667,66.666667,0.000000,2.911000,0.166667,0
1057,92490.0,1.0,1.0,0.0,0.0,0.0,0.266667,0.0,0.166667,0.0,...,1.0,0.000000,0.0,25.766667,1.866667,24.400000,0.000000,2.577667,0.266667,2


In [31]:
df6.to_csv('df6_features.csv', index = False)

# DF7

In [25]:
features7= featurewiz(df77, target='NHY', corr_limit=0.70, verbose=2)
features7

Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
Shape of your Data Set loaded: (1059, 132)
File not able to be loaded
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    131 Predictors classified...
        1 variable(s) removed since they were ID or low-information variables
Shape of your Data Set loaded: (1059, 132)
Number of processors on machine = 1
    No GPU active on this device
    Running XGBoost using CPU parameters
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    131 Predictors classified...
        1 variable(s) removed since they were ID or low-information variables
#### Single_Label Multi_Classification Feature Selection Started ####
Searching for highly correlated variables from 130 variables using SULOV method
#####  SULOV : Searching for Uncorrelat

(['MSEADLG_min',
  'NP3RIGN_max',
  'NP3PSTBL_max',
  'NP2TRMR_max',
  'NP3LGAGR_max',
  'NP3RTCON_var',
  'NP3RIGLU_max',
  'NP2DRES_max',
  'NP3HMOVR_max',
  'NP3RTALU_max',
  'NP3RISNG_max',
  'NP3RTARU_max',
  'NP3RTCON_min',
  'NP3FACXP_min',
  'NP3BRADY_min',
  'NP3GAIT_max',
  'NP3RISNG_min',
  'NP2FREZ_min',
  'NP2HYGN_mean',
  'NP3FRZGT_min',
  'NP3PRSPR_var',
  'NP3BRADY_var',
  'NP3RIGLL_var',
  'NP2HYGN_var',
  'NP3PSTBL_min',
  'NP3FTAPL_min',
  'NP3GAIT_min',
  'NP3TTAPR_min',
  'NP3FTAPR_min',
  'NP3TTAPR_var',
  'MSEADLG_var',
  'NP3LGAGL_var',
  'NP3PRSPL_var',
  'NP3HMOVL_var'],
       MSEADLG_min  NP3RIGN_max  NP3PSTBL_max  NP2TRMR_max  NP3LGAGR_max  \
 0              90            1             0            2             1   
 1              80            0             0            2             2   
 2              85            2             3            1             2   
 3              80            2             0            3             1   
 4              

In [27]:
# Making DAtaframe of Selected columns
columns =['PATNO','MSEADLG_min',
  'NP3RIGN_max',
  'NP3PSTBL_max',
  'NP2TRMR_max',
  'NP3LGAGR_max',
  'NP3RTCON_var',
  'NP3RIGLU_max',
  'NP2DRES_max',
  'NP3HMOVR_max',
  'NP3RTALU_max',
  'NP3RISNG_max',
  'NP3RTARU_max',
  'NP3RTCON_min',
  'NP3FACXP_min',
  'NP3BRADY_min',
  'NP3GAIT_max',
  'NP3RISNG_min',
  'NP2FREZ_min',
  'NP2HYGN_mean',
  'NP3FRZGT_min',
  'NP3PRSPR_var',
  'NP3BRADY_var',
  'NP3RIGLL_var',
  'NP2HYGN_var',
  'NP3PSTBL_min',
  'NP3FTAPL_min',
  'NP3GAIT_min',
  'NP3TTAPR_min',
  'NP3FTAPR_min',
  'NP3TTAPR_var',
  'MSEADLG_var',
  'NP3LGAGL_var',
  'NP3PRSPL_var',
  'NP3HMOVL_var','NHY']
df7= pd.DataFrame(df77, columns=columns)
df7

<IPython.core.display.Javascript object>

,PATNO,MSEADLG_min,NP3RIGN_max,NP3PSTBL_max,NP2TRMR_max,NP3LGAGR_max,NP3RTCON_var,NP3RIGLU_max,NP2DRES_max,NP3HMOVR_max,...,NP3FTAPL_min,NP3GAIT_min,NP3TTAPR_min,NP3FTAPR_min,NP3TTAPR_var,MSEADLG_var,NP3LGAGL_var,NP3PRSPL_var,NP3HMOVL_var,NHY
0,3001,90,1,0,2,1,1.466667,2,1,2,...,0,0,1,1,0.166667,6.666667,0.666667,0.566667,0.166667,2
1,3002,80,0,0,2,2,1.600000,1,2,2,...,2,0,1,1,0.266667,17.500000,0.266667,0.166667,0.000000,2
2,3003,85,2,3,1,2,1.200000,2,1,3,...,1,1,2,2,0.300000,7.500000,0.166667,0.300000,0.266667,3
3,3010,80,2,0,3,1,2.266667,2,2,2,...,2,0,0,0,0.800000,44.166667,0.566667,0.300000,0.400000,2
4,3012,80,1,0,4,2,0.266667,0,2,2,...,1,0,1,1,0.166667,14.166667,0.400000,0.266667,0.266667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236,100,0,0,1,0,0.000000,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0
1055,90456,80,2,1,2,0,0.166667,1,2,1,...,0,1,0,0,0.300000,24.166667,0.166667,0.166667,0.300000,2
1056,91097,100,0,0,0,0,0.000000,0,0,0,...,0,0,0,0,0.166667,0.000000,0.000000,0.000000,0.000000,0
1057,92490,90,0,0,0,1,0.400000,1,1,2,...,0,0,0,0,0.800000,26.666667,0.266667,0.166667,0.166667,2


In [32]:
df7.to_csv('df7_features.csv', index = False)

# df8

In [28]:
features8= featurewiz(df88, target='NHY', corr_limit=0.70, verbose=2)
features8

Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
Shape of your Data Set loaded: (1059, 175)
File not able to be loaded
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    174 Predictors classified...
        2 variable(s) removed since they were ID or low-information variables
Shape of your Data Set loaded: (1059, 175)
Number of processors on machine = 1
    No GPU active on this device
    Running XGBoost using CPU parameters
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    174 Predictors classified...
        2 variable(s) removed since they were ID or low-information variables
#### Single_Label Multi_Classification Feature Selection Started ####
Searching for highly correlated variables from 172 variables using SULOV method
#####  SULOV : Searching for Uncorrelat

(['PARKISM_max',
  'PARKISM_min',
  'GDSWRTLS_max',
  'UPSITBK4_std',
  'STROKE_min',
  'PARKISM_var',
  'BRNINFM_max',
  'PATNO',
  'UPSITBK1_max',
  'GDSBORED_max',
  'GDSWRTLS_min',
  'MCALION_min',
  'UPSITBK1_var',
  'DRMOBJFL_min',
  'DRMUMV_min',
  'MCACUBE_min',
  'MVAWAKEN_min',
  'AGE_ASSESS_SFTANIM_var',
  'UPSITBK3_var',
  'UPSITBK4_var',
  'PTCGBOTH_max',
  'HETRA_min',
  'UPSITBK2_var',
  'AGE_ASSESS_SFTANIM_max',
  'SLPINJUR_min',
  'MCAMONTH_min',
  'MCAPLACE_min',
  'SLPLMBMV_min',
  'GDSENRGY_min',
  'GDSSATIS_min',
  'VLTFRUIT_min',
  'ESS2_var',
  'ESS5_var',
  'ESS5_min',
  'ESS4_min',
  'DRMVERBL_var',
  'DVT_SFTANIM_min',
  'GDSENRGY_var',
  'DEPRS_var',
  'MCAVFNUM_min',
  'ESS8_min'],
       PARKISM_max  PARKISM_min  GDSWRTLS_max  UPSITBK4_std  STROKE_min  \
 0             1.0          1.0           0.0           0.0         0.0   
 1             1.0          1.0           1.0           0.0         0.0   
 2             1.0          1.0           0.0           

In [29]:
# Making DAtaframe of Selected columns
columns =['PATNO','PARKISM_max',
  'PARKISM_min',
  'GDSWRTLS_max',
  'UPSITBK4_std',
  'STROKE_min',
  'PARKISM_var',
  'BRNINFM_max',
  'PATNO',
  'UPSITBK1_max',
  'GDSBORED_max',
  'GDSWRTLS_min',
  'MCALION_min',
  'UPSITBK1_var',
  'DRMOBJFL_min',
  'DRMUMV_min',
  'MCACUBE_min',
  'MVAWAKEN_min',
  'AGE_ASSESS_SFTANIM_var',
  'UPSITBK3_var',
  'UPSITBK4_var',
  'PTCGBOTH_max',
  'HETRA_min',
  'UPSITBK2_var',
  'AGE_ASSESS_SFTANIM_max',
  'SLPINJUR_min',
  'MCAMONTH_min',
  'MCAPLACE_min',
  'SLPLMBMV_min',
  'GDSENRGY_min',
  'GDSSATIS_min',
  'VLTFRUIT_min',
  'ESS2_var',
  'ESS5_var',
  'ESS5_min',
  'ESS4_min',
  'DRMVERBL_var',
  'DVT_SFTANIM_min',
  'GDSENRGY_var',
  'DEPRS_var',
  'MCAVFNUM_min',
  'ESS8_min','NHY']
df8= pd.DataFrame(df88, columns=columns)
df8

<IPython.core.display.Javascript object>

,PATNO,PARKISM_max,PARKISM_min,GDSWRTLS_max,UPSITBK4_std,STROKE_min,PARKISM_var,BRNINFM_max,PATNO,UPSITBK1_max,...,ESS5_var,ESS5_min,ESS4_min,DRMVERBL_var,DVT_SFTANIM_min,GDSENRGY_var,DEPRS_var,MCAVFNUM_min,ESS8_min,NHY
0,3001.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3001.0,4.0,...,0.166667,1.0,0.0,0.166667,46.0,0.300000,0.300000,16.0,0.0,2
1,3002.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,3002.0,3.0,...,0.700000,1.0,1.0,0.166667,46.0,0.000000,0.266667,12.0,0.0,2
2,3003.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3003.0,5.0,...,0.266667,1.0,2.0,0.166667,46.0,0.000000,0.000000,10.0,0.0,3
3,3010.0,1.0,0.0,0.0,0.0,0.0,0.3,0.0,3010.0,2.0,...,0.566667,1.0,0.0,0.166667,36.0,0.000000,0.266667,1.0,0.0,2
4,3012.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3012.0,2.0,...,0.666667,1.0,1.0,0.000000,51.0,0.000000,0.000000,15.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,85236.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85236.0,5.0,...,0.166667,2.0,0.0,0.000000,34.0,0.266667,0.000000,3.0,0.0,0
1055,90456.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90456.0,5.0,...,0.266667,0.0,1.0,0.166667,39.0,0.000000,0.266667,2.0,0.0,2
1056,91097.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,91097.0,7.0,...,0.166667,2.0,1.0,0.000000,44.0,0.000000,0.000000,5.0,0.0,0
1057,92490.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92490.0,3.0,...,0.966667,1.0,0.0,0.266667,69.0,0.266667,0.000000,15.0,0.0,2


In [33]:
df8.to_csv('df8_features.csv', index = False)